In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.

import os
import cv2
import matplotlib.pyplot as plt
import albumentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch
from torch.utils.data import Dataset

from tqdm import tqdm

## Testing

In [ ]:
## 1st place ISIC competition

def get_transforms(image_size):

    transforms_train = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.2, p=0.75),
        albumentations.RandomContrast(limit=0.2, p=0.75),
        albumentations.OneOf([
            albumentations.MotionBlur(blur_limit=5),
            albumentations.MedianBlur(blur_limit=5),
            albumentations.GaussianBlur(blur_limit=5),
            albumentations.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.7),

        albumentations.OneOf([
            albumentations.OpticalDistortion(distort_limit=1.0),
            albumentations.GridDistortion(num_steps=5, distort_limit=1.),
            albumentations.ElasticTransform(alpha=3),
        ], p=0.7),

        albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])
    
    
    transforms_val = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])

    return transforms_train, transforms_val

In [ ]:
print("BEFORE:")
im_gray = cv2.cvtColor(image_or, cv2.COLOR_BGR2GRAY)
print(im_gray.shape)
plt.imshow(im_gray, cmap='gray')

In [ ]:
path = "../input/num110/num/0.png"

image_or = cv2.imread(path)
transforms_train, transforms_val = get_transforms(28)

res = transforms_train(image=image_or)
image = res['image']
print(image.shape)

In [ ]:
print("AFTER:")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),cmap="gray")

# STOPSIGNS

In [ ]:
## 1st place ISIC competition

def get_transforms(image_size):

    transforms_train = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.1, p=0.75),
        albumentations.RandomContrast(limit=0.1, p=0.75),
        albumentations.OneOf([
            albumentations.MotionBlur(blur_limit=3),
            albumentations.MedianBlur(blur_limit=3),
            albumentations.GaussianBlur(blur_limit=2),
            albumentations.GaussNoise(var_limit=(5.0, 15.0)),
        ], p=0.7),

        #albumentations.OneOf([
        #    albumentations.OpticalDistortion(distort_limit=0.2),
        #    albumentations.GridDistortion(num_steps=1, distort_limit=1.),
        #    albumentations.ElasticTransform(alpha=1),
        #], p=0.7),

        #albumentations.CLAHE(clip_limit=1.0, p=0.3),
        #albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.1), max_w_size=int(image_size * 0.1), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])
    
    transforms_val = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])

    return transforms_train, transforms_val

In [ ]:
path = "../input/stopsign/stop.png"

image_or = cv2.imread(path)
transforms_train, transforms_val = get_transforms(30)

res = transforms_train(image=image_or)
image = res['image']
print(image.shape)

In [ ]:
print("BEFORE:")
im_gray = cv2.cvtColor(image_or, cv2.COLOR_BGR2RGB)
print(im_gray.shape)
plt.imshow(im_gray)

In [ ]:
print("AFTER:")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
!mkdir ./stop

In [ ]:
path = "../input/stopsign/stop.png"


for i in range(1000):
    
    image_or = cv2.imread(path)
    transforms_train, transforms_val = get_transforms(30)

    res = transforms_train(image=image_or)
    image = res['image']

    image_x = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_x = (image_x - np.min(image_x))/np.ptp(image_x)
    plt.imsave("./stop/" + str(i) + ".png", image_x)
    
    if i % 50 == 0:
        print(i)

In [ ]:
!zip -r ./stop.zip ./stop

## Modelling

In [84]:
  
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
import timeit

from sklearn.model_selection import train_test_split

EPOCHS = 5
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43 ##############
TEST_SIZE = 0.4


def main(data_path=None, aug_path="../input/vecaugsigns/30w", filename="model.pth", augment=True):
    
    
    np.random.seed(2)
    tf.random.set_seed(2)

    # Check command-line arguments
    if len(sys.argv) not in [2, 3]:
        sys.exit("Usage: python traffic.py data_directory [model.h5]")
    
    # Get image arrays and labels for all image files
    images, labels = load_data(data_path, augment=augment)
    
    # t = timeit.Timer("images, labels = load_data(sys.argv[1])", "from __main__ import load_data")
    # print(t.timeit(number=1))
    
    
    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    
    
    # Add augmented to train
    if augment:
        images, labels = load_vgexp(aug_path)
        # Turn labels into array of shape x, num_categories
        labels = np.zeros((len(labels), NUM_CATEGORIES))
        labels[:, 14] = 1
        x_train = np.concatenate((x_train, np.array(images)), axis=0)
        y_train = np.concatenate((y_train, labels), axis=0)

    # Add original stops to test
    images, labels = load_data(data_path, skip=True, augment=augment)
    # Turn labels into array of shape x, num_categories
    labels = np.zeros((len(labels), NUM_CATEGORIES))
    labels[:, 14] = 1 
    x_test = np.concatenate((x_train, np.array(images)), axis=0)
    y_test = np.concatenate((y_train, labels), axis=0)
    # Alternative, testing only on stop:
    x_test = np.array(images)
    y_test = labels
    

    # Get a compiled neural network
    model = get_model()

    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test, y_test, verbose=2)

    model.summary()

    # Save model to file
    model.save(filename)
    print(f"Model saved to {filename}.")


def load_vgexp(aug_path="../input/vecaugsigns/30w"):
    """
    Loads augmented stop signs to training.
    
    aug_path: path to 1K augmented stop images
    """
    
    images = []
    labels = []
    
    # Do our images work?
    for ppm_file in os.listdir(aug_path):
        raw_ppm = cv2.imread(os.path.join(aug_path, ppm_file))
        resized_ppm = cv2.resize(raw_ppm, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)
        # Append image & label
        images.append(resized_ppm)
        labels.append(14)
                
        
    return (images, labels)
    
    

def load_data(data_dir, skip=False, augment=False):
    """
    Load image data from directory `data_dir`.
    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.
    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    # Declare image & labels list - We later convert the images list into an np array - No we don't!
    # We return a list of NP Arrays!
    # Also we will not label each group of images but EACH image 
    images = []
    labels = []

    # Iterate through each category in directory
    for category_folder in os.listdir(data_dir):
        # Skip .DS_File
        if category_folder.startswith("."):
            continue
            
        if skip:
            # Skip all which are not 14
            if not(category_folder.startswith("14")):
                continue
        else:
            # If augmenting skip 14 in original loading
            if augment==True:
                if category_folder.startswith("14"):
                    continue
            

        # Get the path to each category's folder
        image_folder = os.path.join(data_dir, category_folder)

        # Iterate through the directory of each label
        for ppm_file in os.listdir(image_folder):
            # Extract the ppm file and resize it - No need to focus on the 3 channels here, since imread takes them by default 
            raw_ppm = cv2.imread(os.path.join(image_folder, ppm_file))
            resized_ppm = cv2.resize(raw_ppm, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)
            # Append image & label
            images.append(resized_ppm)
            labels.append(category_folder)
        

    # Return a tuple w/ list of images & list of labels (->43*amount of images per folder labels)
    return (images, labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """

    model = tf.keras.models.Sequential([
        
        # First Convoluational layer to create 32 feature maps with a kernel size of 3,3 (More fm's gave me worse accuracy)
        tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), use_bias=True),

        # Pool by selecting highest values from 2x2 matrix
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

        # Flatten into single dimension
        tf.keras.layers.Flatten(),

        # First Dropout
        tf.keras.layers.Dropout(0.3),

        # Create dense NN - 1 hidden layer is generally enough
        tf.keras.layers.Dense(129, activation="relu"),

        # Second Dropout
        tf.keras.layers.Dropout(0.03),

        # Create Output layer
        tf.keras.layers.Dense(43, activation="softmax")

        ])

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


    return model

In [86]:
# Base
#main("../input/trafficsigns/gtsrb", augment=False)
# Vg augmented
#main("../input/trafficsigns/gtsrb", aug_path="../input/vecaugsigns/30w", augment=True)
# Bitmap augmented
main("../input/trafficsigns/gtsrb", aug_path="../input/vgaug/stop", augment=True)

Epoch 1/5
531/531 [==============================] - 7s 14ms/step - loss: 6.0507 - accuracy: 0.1506
Epoch 2/5
531/531 [==============================] - 7s 14ms/step - loss: 2.7748 - accuracy: 0.2440
Epoch 3/5
531/531 [==============================] - 7s 13ms/step - loss: 2.1859 - accuracy: 0.3543
Epoch 4/5
531/531 [==============================] - 7s 14ms/step - loss: 1.5999 - accuracy: 0.4982
Epoch 5/5
531/531 [==============================] - 8s 15ms/step - loss: 1.2968 - accuracy: 0.5813
17/17 - 0s - loss: 0.0912 - accuracy: 0.9741
Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 

In [72]:
# Make sure both are augmented to 1000 files
!cd ../input/vgaug/stop; ls -1 | wc -l
!cd ../input/vecaugsigns/30w/; ls -1 | wc -l

1000


In [ ]:
Results:
    

Seed 1:
BASE - Original stop shields in test in training & test
Epoch 1/5
500/500 [==============================] - 7s 14ms/step - loss: 2.8991 - accuracy: 0.5000
Epoch 2/5
500/500 [==============================] - 7s 13ms/step - loss: 0.6984 - accuracy: 0.8114
Epoch 3/5
500/500 [==============================] - 7s 14ms/step - loss: 0.4682 - accuracy: 0.8766
Epoch 4/5
500/500 [==============================] - 7s 13ms/step - loss: 0.3476 - accuracy: 0.9107
Epoch 5/5
500/500 [==============================] - 7s 13ms/step - loss: 0.3610 - accuracy: 0.9070
TEST: 17/17 - 0s - loss: 0.0053 - accuracy: 0.9963
        
Aug_VectorGraphics - AugStop shields in training; Original stop shields in test
Epoch 1/5
521/521 [==============================] - 7s 13ms/step - loss: 2.8530 - accuracy: 0.5070
Epoch 2/5
521/521 [==============================] - 7s 14ms/step - loss: 0.6967 - accuracy: 0.8168
Epoch 3/5
521/521 [==============================] - 7s 14ms/step - loss: 0.4639 - accuracy: 0.8782
Epoch 4/5
521/521 [==============================] - 7s 13ms/step - loss: 0.3817 - accuracy: 0.9001
Epoch 5/5
521/521 [==============================] - 7s 14ms/step - loss: 0.3239 - accuracy: 0.9158
TEST: 17/17 - 0s - loss: 5.6308 - accuracy: 0.2333
        
        
Aug_Bitmap - Aug Bitmap stop shields in training; Original stop shields in test
14
Epoch 1/5
521/521 [==============================] - 7s 13ms/step - loss: 2.7158 - accuracy: 0.5508
Epoch 2/5
521/521 [==============================] - 7s 14ms/step - loss: 0.6372 - accuracy: 0.8331
Epoch 3/5
521/521 [==============================] - 7s 14ms/step - loss: 0.4173 - accuracy: 0.8884
Epoch 4/5
521/521 [==============================] - 7s 14ms/step - loss: 0.3833 - accuracy: 0.9034
Epoch 5/5
521/521 [==============================] - 7s 13ms/step - loss: 0.3194 - accuracy: 0.9211
TEST: 17/17 - 0s - loss: 0.2484 - accuracy: 0.9500
Model: "sequential_24"
    

Same order for Seed 2:
    
BASE:
Epoch 1/5
500/500 [==============================] - 7s 13ms/step - loss: 6.0909 - accuracy: 0.1785
Epoch 2/5
500/500 [==============================] - 7s 14ms/step - loss: 1.2191 - accuracy: 0.6929
Epoch 3/5
500/500 [==============================] - 7s 13ms/step - loss: 0.6288 - accuracy: 0.8403
Epoch 4/5
500/500 [==============================] - 7s 14ms/step - loss: 0.4524 - accuracy: 0.8819
Epoch 5/5
500/500 [==============================] - 7s 14ms/step - loss: 0.3456 - accuracy: 0.9090
TEST: 17/17 - 0s - loss: 0.0098 - accuracy: 0.9963
Model: "sequential_27"
    

VectorGraphics:
14
Epoch 1/5
521/521 [==============================] - 7s 13ms/step - loss: 6.0746 - accuracy: 0.1644
Epoch 2/5
521/521 [==============================] - 7s 13ms/step - loss: 2.4584 - accuracy: 0.3501
Epoch 3/5
521/521 [==============================] - 7s 14ms/step - loss: 1.1894 - accuracy: 0.6577
Epoch 4/5
521/521 [==============================] - 8s 14ms/step - loss: 0.7921 - accuracy: 0.7714
Epoch 5/5
521/521 [==============================] - 7s 14ms/step - loss: 0.6189 - accuracy: 0.8222
TEST: 17/17 - 0s - loss: 12.2995 - accuracy: 0.0407
    
BITMAP:
Epoch 1/5
521/521 [==============================] - 7s 14ms/step - loss: 6.1305 - accuracy: 0.1436
Epoch 2/5
521/521 [==============================] - 7s 13ms/step - loss: 2.6992 - accuracy: 0.2950
Epoch 3/5
521/521 [==============================] - 7s 14ms/step - loss: 1.4741 - accuracy: 0.6032
Epoch 4/5
521/521 [==============================] - 7s 14ms/step - loss: 0.8503 - accuracy: 0.7692
Epoch 5/5
521/521 [==============================] - 7s 13ms/step - loss: 0.6088 - accuracy: 0.8328
TEST: 17/17 - 0s - loss: 5.2933 - accuracy: 0.2352
Model: "sequential_25"
